# Tutorial 04 Perturbation Theory

## Introduction

This tutorial demonstrates how perturbation theory is utilized using the
WavesAndEigenvalues package. You may ask: 'What is perturbation theory?'
Well, perturbation theory deals with the approximation of a solution of a
mathematical problem based on a *known* solution of a problem similar to the
problem of interest. (Puhh, that was a long sentence...) The problem is said
to be perturbed from a baseline solution.
You can find a comprehensive presentation of the internal algorithms in
[1,2,3].
## Model set-up and baseline-solution
The model is the same Rijke tube configuration as in Tutorial 01:

In [1]:
using WavesAndEigenvalues.Helmholtz
mesh=Mesh("Rijke_mm.msh",scale=0.001) #load mesh
dscrp=Dict() #initialize model descriptor
dscrp["Interior"]=(:interior, ()) #define resonant cavity
dscrp["Outlet"]=(:admittance, (:Y,1E15)) #specify outlet BC
γ=1.4 #ratio of specific heats
ρ=1.225 #density at the reference location upstream to the flame in kg/m^3
Tu=300.0    #K unburnt gas temperature
Tb=1200.0    #K burnt gas temperature
P0=101325.0 # ambient pressure in Pa
A=pi*0.025^2 # cross sectional area of the tube
Q02U0=P0*(Tb/Tu-1)*A*γ/(γ-1) #the ratio of mean heat release to mean velocity Q02U0
x_ref=[0.0; 0.0; -0.00101] #reference point
n_ref=[0.0; 0.0; 1.00] #directional unit vector of reference velocity
n=0.01 #interaction index
τ=0.001 #time delay
dscrp["Flame"]=(:flame,(γ,ρ,Q02U0,x_ref,n_ref,:n,:τ,n,τ)) #flame dynamics
R=287.05 # J/(kg*K) specific gas constant (air)
speedofsound(x,y,z) = z<0. ? sqrt(γ*R*Tu) : sqrt(γ*R*Tb)
c=generate_field(mesh,speedofsound)
L=discretize(mesh,dscrp,c)

1006×1006-dimensional operator family: 

ω^2*M+K+n*exp(-iωτ)*Q+ω*Y*C

Parameters
----------
n	0.01 + 0.0im
λ	Inf + 0.0im
ω	0.0 + 0.0im
τ	0.001 + 0.0im
Y	1.0e15 + 0.0im


To obtain a baseline solution we solve it using the housholder iteration with
a very small stopping tollerance of `tol=1E-11`.

$$\frac{\textbf{c}\mathcal{Z}}{\rho}$$

In [4]:
L

1006×1006-dimensional operator family: 

ω^2*M+K+n*exp(-iωτ)*Q+ω*Y*C

Parameters
----------
n	0.01 + 0.0im
λ	Inf + 0.0im
ω	0.0 + 0.0im
τ	0.001 + 0.0im
Y	1.0e15 + 0.0im


In [2]:
sol,nn,flag=householder(L,340*2*pi,maxiter=20,tol=1E-11)

Launching Householder...
Iter    Res:     dz:     z:
----------------------------------
0		Inf	Inf	2136.2830044410593
1		1.6243873211323859e6	382.7295713806868	1753.640443755553 + 8.160610349893785im
2		147442.983718491	42.4349412389508	1711.2293969397867 + 9.58445933911067im
3		1804.6396540359215	0.5324135895092171	1710.6978605746078 + 9.615009671129867im
4		0.2839458865676277	8.379743964634571e-5	1710.6977772393602 + 9.615018460179712im
5		8.143682507364851e-8	2.41248261659281e-11	1710.6977772393368 + 9.615018460173921im
6		4.3310085898795686e-8	1.2747836831060415e-11	1710.6977772393495 + 9.615018460173305im
7		1.1278530775405552e-8	3.4155092541817112e-12	1710.6977772393461 + 9.615018460173488im
...finished Householder!
#####################
 Householder results 
#####################
Number of steps: 7
Last step parameter variation:3.4155092541817112e-12
Auxiliary eigenvalue λ residual (rhs):1.1278530775405552e-8
Eigenvalue:1710.6977772393461 + 9.615018460173488im
Eigenvalue/(2*pi):

(####Solution####
eigval:
ω = 1710.6977772393461 + 9.615018460173488im

Parameters:
n = 0.01 + 0.0im
τ = 0.001 + 0.0im
Y = 1.0e15 + 0.0im

Residual:
abs(λ) = 1.1278530775405552e-8, 7, 0)

this small tolerance is necessary because as the name suggests the base-line
solution is the basis to our subsequent steps. If it is inaccurate we will
definetly also encounter inaccurate approximations to other configurations
than the base-line set-up.
## Taylor series

Probably, you are somewhat familiar with the concept of Taylor series
which is a basic example of perturbation theory. There is some function
f from which the value f(x0) is known together with some derivatives at the
same point, the first N say. Then, we can approximate f(x0+Δ) as:

f(x0+Δ)≈∑_n=0^N f_n(x0)Δ^n

Let's consider the time delay `τ`. Our problem depends exponentially on this
value.  We can utilize a fast perturbation algorithm to compute the first
20 Taylor-series coefficients of the eigenfrequency `ω` w.r.t. `τ` by just
typing

In [3]:
perturb_fast!(sol,L,:τ,20)

order: 1, time: 0.37183690071105957
order: 2, time: 0.5182919502258301
order: 3, time: 0.5571959018707275
order: 4, time: 0.601301908493042
order: 5, time: 0.6115520000457764
order: 6, time: 0.6165158748626709
order: 7, time: 0.6420488357543945
order: 8, time: 0.6583240032196045
order: 9, time: 0.6835930347442627
order: 10, time: 0.7013659477233887
order: 11, time: 0.7360508441925049
order: 12, time: 0.7775800228118896
order: 13, time: 0.8279268741607666
order: 14, time: 0.8866078853607178
order: 15, time: 0.9678909778594971
order: 16, time: 1.0668530464172363
order: 17, time: 1.2475378513336182
order: 18, time: 1.4663429260253906
order: 19, time: 1.7090840339660645
order: 20, time: 1.981881856918335


The output shows you how long it takes to compute the coefficients.
Obviously, it takes longer and longer for higher order coefficients.
Note, that there is also an algorithm `perturb!(sol,L,:τ,20)` which does
exactly the same as the fast algorithm but is slower, when it comes to high
orders (N>5).

Both algorithms populate a field in the solution object holding the Taylor
coefficients.

In [4]:
sol.eigval_pert

Dict{Symbol,Any} with 1 entry:
  Symbol("τ/Taylor") => Complex{Float64}[1710.7+9.61502im, 16655.8-1972.54im, -…

In [7]:
sol.eigval_pert[Symbol("τ/Taylor")]

21-element Array{Complex{Float64},1}:
     1710.6977772393461 + 9.615018460173488im
     16655.767682558846 - 1972.5369656026314im
  -1.2612982476560106e6 - 1.46322883044807e7im
   -8.808906967838312e9 + 1.1615582738796182e8im
  -5.873840733061559e11 + 4.175332269859226e12im
  1.7004887170778765e15 + 7.439153396440475e14im
   6.058284950153509e17 - 6.190897825855365e17im
 -1.8938118510974376e20 - 4.003170158460829e20im
 -2.3191390766688626e23 + 2.898858067332429e22im
  -2.275112427776431e25 + 1.2162923188570618e26im
   5.799143769580233e28 + 3.183913640454199e28im
  2.6380804905911956e31 - 2.44451708180038e31im
  -8.222422195967745e33 - 1.8038057337062285e34im
 -1.0957581029026753e37 + 1.211309773723197e36im
 -1.2703381232412134e39 + 6.041956342213871e39im
   3.012726806969943e42 + 1.744584592236548e42im
  1.4744387490957574e45 - 1.3149324622042034e45im
 -4.5028654465230355e47 - 1.0334597113739691e48im
  -6.434729914382895e50 + 6.048576186744849e49im
  -8.243537285189499e52 + 3.6288343

We can use these values to form the Taylor-series approximation and for
convenience we can just do so by calling to the solution object itself.
For instance let's assume we would like to approximate the value of the
eigenfrequency when "τ==0.00125" based on the 20th order series expansion.

In [5]:
Δ=0.00001
ω_approx=sol(:τ,τ+Δ,20)

1710.8641999717368 + 9.593830019669932im

Let's compare this value to the true solution:

In [6]:
L.params[:τ]=τ+Δ #change parameter
sol_exact,nn,flag=householder(L,340*2*pi,maxiter=20,tol=1E-11) #solve again
ω_exact=sol_exact.params[:ω]
println(" exact=$(ω_exact/2/pi)  vs  approx=$(ω_approx/2/pi))")

Launching Householder...
Iter    Res:     dz:     z:
----------------------------------
0		Inf	Inf	2136.2830044410593
1		1.623778519161384e6	382.57794209467136	1753.7917372224454 + 8.143235161319367im
2		147393.3700244174	42.419892790390364	1711.395624822153 + 9.563431517456596im
3		1803.9831279743273	0.5322099000437782	1710.8642832754622 + 9.593821293119722im
4		0.283822856662201	8.375955651404198e-5	1710.864199971736 + 9.593830019661011im
5		6.251839010789878e-8	1.8483777533157128e-11	1710.8641999717524 + 9.593830019669593im
6		1.2566139001166132e-8	3.677060802916394e-12	1710.864199971756 + 9.593830019670127im
...finished Householder!
#####################
 Householder results 
#####################
Number of steps: 6
Last step parameter variation:3.677060802916394e-12
Auxiliary eigenvalue λ residual (rhs):1.2566139001166132e-8
Eigenvalue:1710.864199971756 + 9.593830019670127im
Eigenvalue/(2*pi):272.2924943844659 + 1.5269054708139163im
 exact=272.2924943844659 + 1.5269054708139163im 

Clearly, the approximation matches the first 4 digits after the point!

We can also compute the approximation at any lower order than 20.
For instance, the first-order approximation is:

In [8]:
ω_approx=sol(:τ,τ+Δ,1)
println(" first-order approx=$(ω_approx/2/pi)")

 first-order approx=272.4515667269969 + 1.508301985260934im


Note that the accuracy is less than at twentieth order.

Also note that we cannot compute the perturbation at higher order than 20
because we have only computed the Taylor series coefficients up to 20th order.
Of course we could, prepare higher order coefficients, let's say up to 30th
order by

In [9]:
perturb_fast!(sol,L,:τ,30)

order: 1, time: 0.022557973861694336
order: 2, time: 0.028818130493164062
order: 3, time: 0.03693509101867676
order: 4, time: 0.04986906051635742
order: 5, time: 0.07743716239929199
order: 6, time: 0.10853910446166992
order: 7, time: 0.13708996772766113
order: 8, time: 0.18765902519226074
order: 9, time: 0.24693608283996582
order: 10, time: 0.3387761116027832
order: 11, time: 0.44678807258605957
order: 12, time: 0.6233570575714111
order: 13, time: 0.872495174407959
order: 14, time: 1.2278120517730713
order: 15, time: 1.6134400367736816
order: 16, time: 2.115550994873047
order: 17, time: 2.6285011768341064
order: 18, time: 3.3662450313568115
order: 19, time: 4.238720178604126
order: 20, time: 5.417253017425537
order: 21, time: 6.968240976333618
order: 22, time: 9.01108717918396
order: 23, time: 11.619636058807373
order: 24, time: 15.123805046081543
order: 25, time: 19.599322080612183
order: 26, time: 25.285834074020386
order: 27, time: 32.59842801094055
order: 28, time: 42.1113080978393

and then

In [10]:
ω_approx=sol(:τ,τ+Δ,30)
println(" 30th-order approx=$(ω_approx/2/pi)")

 30th-order approx=272.4512688913611 + 1.5065029271777874im


Indeed, `30` is a special limit because per default the WavesAndEigenvalues
package installs the perturbation algorithm on your machine up to 30th order.
This is because higher orders would consume significantly more memory in
the installation directory and also the computation of higher orders may take
some time.  However, if you really want to use higher orders. You can rebuild
your package by first setting a special environment variable to your desired
order -- say 40 -- by `ENV["JULIA_WAE_PERT_ORDER"]=40` and then run
`Pkg.build("WavesAndEigenvalues")`
Note, that if you don't make the environment variable permanent,
these steps will be necessary once every time you got any update to the
package from julias package manager.

In [11]:
#TODO: Padé, speed, conv_radius

Ok, we've seen how we can compute Taylor-series coefficients and how to
evaluate them as a Taylor series. But how do we choose parameters like
the baseline set-up or the eprturbation order to get a reasonable estimate?
Well there is a lot you can do but, unfornately, there are a lot of
misunderstandings when it comes to the quality perturbative approximations.

Take a second and try to answer the following questions:
1. What is the range of Δ in which you can expect reliable results from
the approximation, i.e., the difference from the true solution is small?
2. How does the quality of your approximation improve if you increase the
number of known derivatives?
3. What else can you do to improve your solution?

Regarding the first question, many people misbelieve that the approximation is
good as long as Δ (the shift from the expansion point) is small. This is right
and wrong at the same time. Indeed, to classify Δ as small, you need to
compare it agains some value. For Taylor series this is the radius of
convergence. Convergence radii can be quite huge and sometimes super small.
Also keep in mind that the nuemrical value of Δ in most engineering
applications is meaningless if it is not linked to some unit of measure.
(You know the joke: What is larger, 1 km or a million mm?)

So how do we get the radius of convergence? It's as simple like

In [12]:
r=conv_radius(sol,:τ)
#The return value here is an array that holds N-1 values, where N is the order

30-element Array{Float64,1}:
 0.014571122174149822
 0.0010248272332944286
 0.0014999584244220145
 0.0020289441884251895
 0.0016357263023061576
 0.000708225140874304
 0.0008838641300182804
 0.0011994157853873137
 0.0015466591161431436
 0.001009701567329566
 0.0007140149599831477
 0.000922996311971907
 0.0011524432243893966
 ⋮
 0.0010724059790236929
 0.0009194340436461991
 0.0008946495288799059
 0.0009724034710172647
 0.0011447898954301161
 0.0010453439687707167
 0.0008677597554851435
 0.0009263400643087007
 0.0009868297577412155
 0.0010468111645774323
 0.0010306624013915356
 0.0008798274754933992

of Taylor-series coefficients that have been computed for `:τ`. This is
because the convergenvce radius is estimated based on the ratio of two
consecutive Taylor-series coefficients. Usually, the last entry of r should
give you the best approximate.

In [13]:
println("Best estimate available: r=$(r[end])")

Best estimate available: r=0.0008798274754933992


However, there are cases where the estimation procedure for the convergence
radius is not appropriate. That is why you can have a look on the other entries
in r to judge whether there is smooth convergence.
Let's see what's happening if we evaluate the Taylor series beyond it's radius
of convergence.

In [14]:
ω_approx=sol(:τ,τ+r[end]+0.001,20)
L.params[:τ]=τ+r[end]+0.001
sol_exact,nn,flag=householder(L,340*2*pi,maxiter=20,tol=1E-11) #solve again
ω_exact=sol_exact.params[:ω]
println(" exact=$(ω_exact/2/pi)  vs  approx=$(ω_approx/2/pi))")

Launching Householder...
Iter    Res:     dz:     z:
----------------------------------
0		Inf	Inf	2136.2830044410593
1		1.6737160045312347e6	390.56818561491536	1745.8261131206536 - 9.323284557056754im
2		135881.95640391385	37.975308809095104	1707.8511237381867 - 9.167526579212753im
3		1393.9604180540105	0.3976932811125874	1707.456569224424 - 9.11765994195433im
4		0.15434691496937444	4.404423515376954e-5	1707.4565281774587 - 9.117643971941812im
5		5.699345277176264e-9	1.5559141903449318e-12	1707.4565281774599 - 9.11764397194075im
...finished Householder!
#####################
 Householder results 
#####################
Number of steps: 5
Last step parameter variation:1.5559141903449318e-12
Auxiliary eigenvalue λ residual (rhs):5.699345277176264e-9
Eigenvalue:1707.4565281774599 - 9.11764397194075im
Eigenvalue/(2*pi):271.75014657396883 - 1.4511181074863926im
 exact=271.75014657396883 - 1.4511181074863926im  vs  approx=-777965.8462191146 - 783623.5455555173im)


Outch, the approximation is completely off.
Remember question 2? Maybe the estimate gets better if we increase the
perturbation order ? At least the last time we've seen an improvement.
But this time...

In [14]:
ω_approx=sol(:τ,τ+r[end]+0.001,30)
println(" exact=$(ω_exact/2/pi)  vs  approx=$(ω_approx/2/pi))")

 exact=271.75014657396883 - 1.4511181074863926im  vs  approx=3.9380584235293037e8 - 6.32156174019385e8im)


things get *way' worse! This is exaclty the reason why some clever person
named r the radius of *convergence*. Beyond that radius we cannot make the
Taylor-series converge without shifting the expansion point. You might think:
'Well, then let's shift the expansion point!' While this is definitely a valid
approach, there is something better you can do...

## Series accelartion by Padé approximation

The Taylor-series cannot converge beyond its radius of convergence. So why
not trying someting else than a Taylor-series?. The truncated Taylor series is
a polynomial approximation to our unknown relation ω=ω(τ). An alternative
(if not to say a generalazation) of this approach is to consider rational
polynomial approximations.  So isntead of
f(x0+Δ)≈∑_n=0^N f_n(x0)Δ^n
we try something like
f(x0+Δ)≈[ ∑_l=0^L a_l(x0)Δ^l ] / [ 1 + ∑_m=0^M b_m(x0)Δ^m ]
In order to get the same asymptotic behaviour close to our expansion point, we
demand that the truncated Taylor series expansion of our Padé approximant is
identical to the approximation of our unknown function. Here comes the clou
we already know these values because we computed the Taylor-series
coefficients. Only little algebra is needed to convert the Taylor coefficients
to Padé coefficients and all of this is build in to solution type. All you
need to know is that the number of Padé coefficients is related to the number
of Taylor coefficients by the simple formula L+M=N. Let's give it a try and
compute the Padé approximant for L=M=10 (a so-called diagonal Padé
approximant). This is just achieve by an extra argument for the call to our
solution object.

In [15]:
ω_approx=sol(:τ,τ+r[end]+0.001,10,10)
println(" exact=$(ω_exact/2/pi)  vs  approx=$(ω_approx/2/pi))")

 exact=271.75014657396883 - 1.4511181074863926im  vs  approx=272.38521958672044 - 15.664405510782343im)


Wow! There is now only a diffrence of about 1hz between the true solution and
it's estimate. I'would say that's fine for many egineering applications.
What's your opinion?
## Conclusion
You learned how to use perturbation theory. The main computational costs for
this approach is the computation of Taylor-series coefficients. Once you have
them everything comes down to ta few evaluations of scalar polynomials.
Especially, when you like to rapidly evalaute your model for a bunch of values
in a given parameter range. This approach should speed up your computations.

The next tutorial will familiarize you with the use of higher order
finite elements.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*